In [ ]:
from kafka import KafkaConsumer
import json

In [ ]:
c = KafkaConsumer('movielens_ratings' , bootstrap_servers=['localhost:9092'])

In [ ]:
def process_msg(msg):
    print(msg.offset)
    dico = dict(json.loads(msg.value))
    print(dico)

In [ ]:
for msg in c:
    process_msg(msg)

In [ ]:
from kafka import KafkaConsumer
import json

consumer = KafkaConsumer(
    'MoviesRatings',
    bootstrap_servers=['localhost:9092'],
    auto_offset_reset='earliest',  # Pour lire depuis le début
    group_id='my-group',  # Identifiant de groupe de consommateurs
    value_deserializer=lambda x: json.loads(x.decode('utf-8'))  # Pour désérialiser automatiquement
)

for message in consumer:
    print(f"Offset: {message.offset}")
    print(f"Valeur: {message.value}")

In [ ]:
!pip install numpy


In [ ]:
!pip install pymongo


In [ ]:
# streaming_recommendations.py
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_json, explode, array, lit, to_json, struct
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType, TimestampType
from pyspark.ml.recommendation import ALSModel
import time
import json
from pymongo import MongoClient
import datetime

# Créer une session Spark avec support Kafka et MongoDB
spark = SparkSession.builder \
    .appName("MovieRecommendationStreaming") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2,org.mongodb.spark:mongo-spark-connector_2.12:3.0.1") \
    .getOrCreate()

# Définir le schéma des données entrantes
schema = StructType([
    StructField("userId", IntegerType(), True),
    StructField("movieId", IntegerType(), True),
    StructField("rating", DoubleType(), True),
    StructField("timestamp", IntegerType(), True)
])

# Charger le modèle ALS préalablement entraîné
model_path = "hdfs://namenode:9000/models/als"
model = ALSModel.load(model_path)


# Configuration MongoDB
mongo_uri = "mongodb://localhost:27017"
mongo_client = MongoClient(mongo_uri)
mongo_db = mongo_client["movie_recommender"]
mongo_collection = mongo_db["recommendations"]

# Fonction pour générer des recommandations et les sauvegarder dans MongoDB
def process_batch(df, epoch_id):
    if not df.isEmpty():
        try:
            # Extraire les userId uniques du batch
            unique_users = df.select("userId").distinct()
            
            # Générer des recommandations pour chaque utilisateur
            recommendations = model.recommendForUserSubset(unique_users, 10)
            
            # Exploser les recommandations pour avoir un format plat
            flat_recommendations = recommendations.select(
                col("userId"),
                explode(col("recommendations")).alias("rec")
            ).select(
                col("userId"),
                col("rec.movieId").alias("movieId"),
                col("rec.rating").alias("prediction")
            )
            
            # Joindre avec les informations de films (si disponible)
            try:
                movies_df = spark.read.csv("hdfs://namenode:9000/datasets/movies.csv", header=True)
                recommendations_with_info = flat_recommendations.join(
                    movies_df, flat_recommendations.movieId == movies_df.movieId
                ).select(
                    flat_recommendations.userId,
                    flat_recommendations.movieId,
                    movies_df.title,
                    flat_recommendations.prediction
                )
            except:
                recommendations_with_info = flat_recommendations
            
            # Afficher les recommandations
            recommendations_with_info.show(10, False)
            
            # Convertir le DataFrame en liste de dictionnaires pour MongoDB
            recommendations_list = recommendations_with_info.withColumn(
                "timestamp", lit(datetime.datetime.now().isoformat())
            ).toJSON().collect()
            
            # Insérer dans MongoDB
            if recommendations_list:
                documents = [json.loads(rec) for rec in recommendations_list]
                mongo_collection.insert_many(documents)
                
            print(f"Batch {epoch_id}: Recommandations générées et sauvegardées pour {unique_users.count()} utilisateurs")
        
        except Exception as e:
            print(f"Erreur lors du traitement du batch {epoch_id}: {str(e)}")

# Lire les données du stream Kafka
kafka_stream = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "MoviesRatings") \
    .option("startingOffsets", "latest") \
    .load()

# Transformer les données JSON en DataFrame structuré
parsed_stream = kafka_stream \
    .select(from_json(col("value").cast("string"), schema).alias("data")) \
    .select("data.*")

# Traiter les données par batch
query = parsed_stream \
    .writeStream \
    .foreachBatch(process_batch) \
    .outputMode("update") \
    .trigger(processingTime="10 seconds") \
    .start()

# Attendre la fin du traitement
query.awaitTermination()

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_json, explode, lit
from pyspark.ml.recommendation import ALSModel
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType, StringType
import time
import json
from pymongo import MongoClient
import datetime
import os
import socket


# Créer une session Spark avec support Kafka et MongoDB
spark = SparkSession.builder \
    .appName("MovieRecommendationStreaming") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2,org.mongodb.spark:mongo-spark-connector_2.12:3.0.1") \
    .getOrCreate()

# Définir le schéma des données entrantes
schema = StructType([
    StructField("userId", IntegerType(), True),
    StructField("movieId", IntegerType(), True),
    StructField("rating", DoubleType(), True),
    StructField("timestamp", IntegerType(), True)
])

# Configuration des services - Permettre de spécifier via variables d'environnement ou arguments
# Pour MongoDB, nous utilisons localhost comme demandé
KAFKA_SERVER = os.environ.get("KAFKA_SERVER", "localhost:9092")  # Changé de kafka:9092 à localhost:9092
MONGO_URI = "mongodb://localhost:27017"  # Fixé à localhost comme demandé
HDFS_HOST = os.environ.get("HDFS_HOST", "namenode:9000")

# Afficher les configurations pour débogage
print(f"Connexion Kafka: {KAFKA_SERVER}")
print(f"Connexion MongoDB: {MONGO_URI}")
print(f"Connexion HDFS: {HDFS_HOST}")

try:
    # Charger le modèle ALS préalablement entraîné
    model_path = f"hdfs://{HDFS_HOST}/models/als"
    print(f"Tentative de chargement du modèle depuis: {model_path}")
    model = ALSModel.load(model_path)
    print("Modèle chargé avec succès!")
except Exception as e:
    print(f"Erreur lors du chargement du modèle: {str(e)}")
    # On continue même si le modèle n'est pas chargé - nous gérerons ce cas dans process_batch

# Configuration MongoDB
try:
    print(f"Tentative de connexion à MongoDB: {MONGO_URI}")
    mongo_client = MongoClient(MONGO_URI, serverSelectionTimeoutMS=5000)
    # Vérifier la connexion
    mongo_client.server_info()
    print("Connexion à MongoDB établie avec succès!")
    
    mongo_db = mongo_client["movie_recommender"]
    mongo_collection = mongo_db["recommendations"]
except Exception as e:
    print(f"Erreur de connexion à MongoDB: {str(e)}")
    print("Le programme continuera sans MongoDB - les recommandations ne seront pas sauvegardées.")

# Fonction pour vérifier la connexion MongoDB
def check_mongo_connection():
    try:
        # Vérifier la connexion à MongoDB
        mongo_client.server_info()
        return True
    except Exception as e:
        print(f"Erreur de connexion à MongoDB: {str(e)}")
        return False

# Fonction pour générer des recommandations et les sauvegarder dans MongoDB
def process_batch(df, epoch_id):
    if df.isEmpty():
        print(f"Batch {epoch_id}: Aucune donnée reçue.")
        return
        
    try:
        print(f"Traitement du batch {epoch_id}. Nombre d'enregistrements: {df.count()}")
        
        # Extraire les userId uniques du batch
        unique_users = df.select("userId").distinct()
        unique_count = unique_users.count()
        print(f"Nombre d'utilisateurs uniques dans ce batch: {unique_count}")
        
        if unique_count == 0:
            print("Aucun utilisateur à traiter dans ce batch.")
            return
        
        # Générer des recommandations pour chaque utilisateur
        try:
            recommendations = model.recommendForUserSubset(unique_users, 10)
            print(f"Recommandations générées pour {recommendations.count()} utilisateurs")
        except Exception as e:
            print(f"Erreur lors de la génération des recommandations: {str(e)}")
            return
        
        # Exploser les recommandations pour avoir un format plat
        flat_recommendations = recommendations.select(
            col("userId"),
            explode(col("recommendations")).alias("rec")
        ).select(
            col("userId"),
            col("rec.movieId").alias("movieId"),
            col("rec.rating").alias("prediction")
        )
        
        # Joindre avec les informations de films (si disponible)
        try:
            movies_path = f"hdfs://{HDFS_HOST}/datasets/movie.csv"
            print(f"Tentative de chargement des informations des films depuis: {movies_path}")
            movies_df = spark.read.csv(movies_path, header=True)
            recommendations_with_info = flat_recommendations.join(
                movies_df, flat_recommendations.movieId == movies_df.movieId
            ).select(
                flat_recommendations.userId,
                flat_recommendations.movieId,
                movies_df.title,
                flat_recommendations.prediction
            )
            print("Informations des films chargées et jointes avec succès")
        except Exception as e:
            print(f"Erreur lors du chargement des informations des films: {str(e)}")
            print("Continuation sans les informations des films")
            recommendations_with_info = flat_recommendations
        
        # Afficher les recommandations pour débogage
        print("Exemple de recommandations générées:")
        recommendations_with_info.show(5, False)
        
        # Sauvegarder dans MongoDB si disponible
        if check_mongo_connection():
            try:
                # Convertir le DataFrame en liste de dictionnaires pour MongoDB
                recommendations_list = recommendations_with_info.withColumn(
                    "timestamp", lit(datetime.datetime.now().isoformat())
                ).toJSON().collect()
                
                # Insérer dans MongoDB
                if recommendations_list:
                    documents = [json.loads(rec) for rec in recommendations_list]
                    mongo_collection.insert_many(documents)
                    print(f"Données sauvegardées dans MongoDB pour {len(documents)} recommandations")
            except Exception as e:
                print(f"Erreur lors de la sauvegarde dans MongoDB: {str(e)}")
        else:
            print("MongoDB non disponible, les recommandations ne sont pas sauvegardées")
        
        print(f"Traitement du batch {epoch_id} terminé avec succès!")
    
    except Exception as e:
        print(f"Erreur générale lors du traitement du batch {epoch_id}: {str(e)}")

# Ajouter une fonction pour tester la connexion à Kafka
def test_kafka_connection():
    try:
        print(f"Test de connexion à Kafka sur {KAFKA_SERVER}...")
        # Essayer un test basique de connexion au socket
        host, port = KAFKA_SERVER.split(':')
        socket_test = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        socket_test.settimeout(5)
        result = socket_test.connect_ex((host, int(port)))
        socket_test.close()
        
        if result == 0:
            print(f"Port {port} sur {host} est ouvert (socket test).")
            # Le test socket est réussi, mais continuons quand même
            
        # On continue sans tester avec KafkaAdminClient pour éviter les erreurs
        # Nous verrons si la connexion fonctionne lors de la création du stream
        return True
    except Exception as e:
        print(f"Erreur lors du test de connexion à Kafka (socket): {str(e)}")
        # On continue quand même - le problème peut être spécifique au test
        return True

# Tester la connexion à Kafka avant de démarrer le stream
test_kafka_connection()
print("Tentative de démarrage du stream Kafka même si le test de connexion échoue...")

# Lire les données du stream Kafka avec plus d'options de configuration
try:
    print(f"Initialisation du stream Kafka depuis {KAFKA_SERVER}...")
    kafka_stream = spark.readStream \
        .format("kafka") \
        .option("kafka.bootstrap.servers", KAFKA_SERVER) \
        .option("subscribe", "MoviesRatings") \
        .option("startingOffsets", "latest") \
        .option("kafka.security.protocol", "PLAINTEXT") \
        .option("failOnDataLoss", "false") \
        .option("kafkaConsumer.pollTimeoutMs", "5000") \
        .load()
    
    print("Stream Kafka initialisé avec succès!")
    
    # Transformer les données JSON en DataFrame structuré
    parsed_stream = kafka_stream \
        .select(from_json(col("value").cast("string"), schema).alias("data")) \
        .select("data.*")
    
    # Traiter les données par batch avec plus de robustesse
    query = parsed_stream \
        .writeStream \
        .foreachBatch(process_batch) \
        .outputMode("update") \
        .option("checkpointLocation", "/tmp/checkpoint") \
        .trigger(processingTime="10 seconds") \
        .start()
    
    print("Stream de traitement démarré! En attente de données...")
    
    # Attendre la fin du traitement avec gestion des exceptions
    query.awaitTermination()
    
except Exception as e:
    print(f"Erreur lors du démarrage du stream: {str(e)}")
    print("Si l'erreur persiste, essayez d'exécuter ce script avec une adresse Kafka différente:")
    print("KAFKA_SERVER=host.docker.internal:9092 python MovieRecommendationStreaming.py")
    print("ou trouvez l'adresse IP du broker Kafka et utilisez:")
    print("KAFKA_SERVER=172.17.0.x:9092 python MovieRecommendationStreaming.py")

Connexion Kafka: localhost:9092
Connexion MongoDB: mongodb://localhost:27017
Connexion HDFS: namenode:9000
Tentative de chargement du modèle depuis: hdfs://namenode:9000/models/als
Modèle chargé avec succès!
Tentative de connexion à MongoDB: mongodb://localhost:27017
Erreur de connexion à MongoDB: localhost:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 5.0s, Topology Description: <TopologyDescription id: 6817d61b07dd3e759ae8be8b, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>
Le programme continuera sans MongoDB - les recommandations ne seront pas sauvegardées.
Test de connexion à Kafka sur localhost:9092...
Port 9092 sur localhost est ouvert (socket test).
Tentative de démarrage du stream Kafka m

25/05/04 21:03:29 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/05/04 21:03:29 WARN AdminClientConfig: The configuration 'key.deserializer' was supplied but isn't a known config.
25/05/04 21:03:29 WARN AdminClientConfig: The configuration 'value.deserializer' was supplied but isn't a known config.
25/05/04 21:03:29 WARN AdminClientConfig: The configuration 'enable.auto.commit' was supplied but isn't a known config.
25/05/04 21:03:29 WARN AdminClientConfig: The configuration 'max.poll.records' was supplied but isn't a known config.
25/05/04 21:03:29 WARN AdminClientConfig: The configuration 'auto.offset.reset' was supplied but isn't a known config.


Stream de traitement démarré! En attente de données...


25/05/04 21:03:38 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
25/05/04 21:03:45 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 10000 milliseconds, but spent 14503 milliseconds
25/05/04 21:03:55 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 10000 milliseconds, but spent 10022 milliseconds
25/05/04 21:04:06 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 10000 milliseconds, but spent 10758 milliseconds
25/05/04 21:04:53 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 10000 milliseconds, but spent 10032 milliseconds
25/05/04 21:05:40 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 10000 milliseconds, but spent 10350 milliseconds
25/05/04 21:06:10 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 10000 milliseconds, but 

Erreur lors du démarrage du stream: [STREAM_FAILED] Query [id = 1ce21465-9b19-42c3-80c7-2eeb15d62be4, runId = b4f1793c-f9e7-4def-88dd-0b863fced500] terminated with exception: An exception was raised by the Python Proxy. Return Message: Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 617, in _call_proxy
    return_value = getattr(self.pool[obj_id], method)(*params)
  File "/usr/local/lib/python3.10/dist-packages/pyspark/sql/utils.py", line 120, in call
    raise e
  File "/usr/local/lib/python3.10/dist-packages/pyspark/sql/utils.py", line 117, in call
    self.func(DataFrame(jdf, wrapped_session_jdf), batch_id)
  File "/tmp/ipykernel_48348/2072420800.py", line 74, in process_batch
    if df.isEmpty():
  File "/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py", line 883, in isEmpty
    return self._jdf.isEmpty()
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1322, in __call__
    re